In [1]:
import pandas as pd
import numpy as np
import spacy
import nltk

In [2]:
df_contenido = pd.read_csv('../data/curated/contenido_procesado.csv', encoding='utf-8-sig', index_col=[0])

FileNotFoundError: [Errno 2] No such file or directory: '../data/curated/contenido_procesado.csv'

In [29]:
df_curated = pd.read_csv('../data/curated/curated_database.csv', encoding='utf-8-sig', index_col=[0])

In [ ]:
df = pd.merge(df_curated, df_contenido, left_index=True, right_index=True)

In [ ]:
df = df.drop('Contenido_y', axis=1)
df.rename(columns = {'Contenido_x': 'Contenido'}, inplace = True)

In [ ]:
df.to_csv('../data/curated/ensayo.csv', encoding='utf-8-sig')

In [ ]:
entidades = set()
for lista in df['Entidades de Contenido'].values:
    if type(lista) is str:
        aux = lista.split(', ')
        for palabra in aux:
            entidades.add(palabra)

In [ ]:
palabras = []
for i, lista in enumerate(df['Contenido procesado'].values):
    if type(lista) is str:
        aux = lista.split(', ')
        for palabra in aux:
            if palabra in entidades:
                palabras.append([i, palabra, 1])
            else:
                palabras.append([i, palabra, 0])   

In [ ]:
def get_apariciones(entidades, num=False):
    '''Crear un diccionario de diccionarios para rastrear las coapariciones
    de entidades.
    Parameters
    ----------
    entidades: lista de las entidades
    num       : si el diccionario de diccionarios debe o no inicializarse en 0 
    (True) o en una lista vacía (False)
    Returns
    -------
    Diccionario de diccionarios donde cada clave es cada entidad y cada valor
    es un diccionario con todos las demás entidades como claves, y valores como 
    0 o [].
    '''
    
    apariciones = {}
    for entidad in entidades:
        relaciones = {}
        for relacion in entidades:
            if relacion != entidad:

                if num:
                    relaciones[relacion] = 0

                else:
                    relaciones[relacion] = []
        apariciones[entidad] = relacion
    return apariciones

In [ ]:
count = 0
for i, lists in enumerate(df['Contenido'].values):
    if type(lists) is float:
        print(i)

In [ ]:
df[df.Contenido.str.len() < 30]

,Fecha Extraccion,Titulo,Fecha Publicacion,Tema,URL,Imagen,Empresa,Fuente,Resumen,Autor,Contenido,RelNewsUrls,Tags,Contenido procesado,Contenido segmentado,Entidades de Contenido
77,03-02-2023,"Lo que tiene que ver Petro, y lo que no, con e...",15-07-2022,NaN,https://www.lasillavacia.com/historias/silla-n...,NaN,ecopetrol,La Silla Vacía,NaN,La Silla Vacía,,NaN,NaN,NaN,,NaN
100,03-02-2023,Ecopetrol también quiere opinar,30-05-2021,NaN,https://www.lasillavacia.com/historias/silla-n...,NaN,ecopetrol,La Silla Vacía,NaN,NaN,,NaN,NaN,NaN,,NaN


In [ ]:
df = df[df['Contenido procesado'].notna()]

In [ ]:
def lista_palabras():
    entidades = set()
    for lista in df[f'Entidades de {columna}'].values:
        if type(lista) is str:
            aux = lista.split(', ')
            for palabra in aux:
                entidades.add(palabra)

    palabras = []
    for i in df.index:
        lista = df.loc[i, 'Contenido procesado']
        if type(lista) is str:
            aux = lista.split(', ')
            for palabra in aux:
                if palabra in entidades:
                    palabras.append([i, palabra, 1])
                else:
                    palabras.append([i, palabra, 0])

In [ ]:
def lista_palabras(val_ent, val_pal, indice):
    entidades = set(val_ent.split(', '))

    palabras = val_pal.split(', ')
    freq_pal = dict(nltk.FreqDist(palabras))
    
    lista = []
    for key, value in freq_pal.items():
        if key in entidades:
            lista.append([key, value, indice, 1])
        else:
            lista.append([key, value, indice, 0])
            
    return pd.DataFrame(lista, columns=['Palabra', 'Frecuencia', 'ID_Articulo', 'Entidad'])
    

In [ ]:
lista_palabras(df.loc[0, 'Entidades de Contenido'], df.loc[0, 'Contenido procesado'], 0)

,Palabra,Frecuencia,ID_Articulo,Entidad
0,movimiento,2,0,0
1,dólar,8,0,0
2,semanas,1,0,0
3,atrás,1,0,0
4,venía,1,0,0
...,...,...,...,...
230,aire,1,0,0
231,Alexander,1,0,0
232,Ríos,1,0,0
233,fundador,1,0,0


In [ ]:
df_raw = pd.read_csv('../data/raw/database.csv',
                     encoding='utf-8-sig', index_col=[0])
df_curated = pd.read_csv(
    '../data/curated/curated_database.csv', encoding='utf-8-sig', index_col=[0])
len_curated = len(df_curated)
# Verificar cuales articulos no han sido procesados
df = df_raw[~df_raw['Titulo'].isin(df_curated['Titulo'])].reset_index()
len_df = len(df)
df

,index,Fecha Extraccion,Titulo,Fecha Publicacion,Tema,URL,Imagen,Empresa,Fuente,Resumen,Autor,Contenido,RelNewsUrls,Tags
0,63,2023-02-03 14:51:13,"Felipe Bayón, difícil de reemplazar en la pres...",2023-01-23 00:00:00,Confidenciales,https://www.semana.com/confidenciales/articulo...,https://www.semana.com/resizer/vfF62WvmBdVguGl...,ecopetrol,Semana,Muchos líderes de opinión en el país coinciden...,NaN,SIN PARRAFOS,NaN,"['Confidenciales Semana', 'Felipe Bayón Pardo'..."
1,78,2023-02-03 14:54:41,"Lo que tiene que ver Petro, y lo que no, con e...",2022-07-15 00:00:00,NaN,https://www.lasillavacia.com/historias/silla-n...,NaN,ecopetrol,La Silla Vacía,NaN,La Silla Vacía,,NaN,NaN
2,101,2023-02-03 14:54:43,Ecopetrol también quiere opinar,2021-05-30 00:00:00,NaN,https://www.lasillavacia.com/historias/silla-n...,NaN,ecopetrol,La Silla Vacía,NaN,NaN,,NaN,NaN
3,166,2023-02-03 15:26:54,Entre iguanas,2023-01-28 00:00:00,CARICATURAS,https://www.eltiempo.com/opinion/caricaturas/r...,NaN,ecopetrol,El Tiempo,Caricatura de Guerreros,NaN,SIN PARRAFOS,NaN,NaN
4,249,2023-02-09 13:59:34,La radio local fue premiada en España,2018-11-15 00:00:00,CAJA FUERTE,https://www.larepublica.co/caja-fuerte/la-radi...,"data:image/svg+xml,%3Csvg xmlns='http://www.w3...",caracol,La República,NaN,SIN AUTOR,SIN PARRAFOS,[],NaN
5,302,2023-02-03 14:50:43,Este título es una prueba,2023-03-03,BOLSAS,https://www.google.com/,NaN,ensayo,ensayo,Este es el resumen del ensayo,ANDRÉS OSPINA PATIÑO,Acá debo escribir un contenido que se al menos...,NaN,NaN


In [23]:
val_ent = df_curated.loc[0, 'Entidades de Contenido']
val_pal = df_curated.loc[0, 'Contenido procesado']

In [24]:
df_palabras = pd.read_csv('../data/curated/palabras.csv', encoding='utf-8-sig', index_col=[0])

In [45]:
def lista_ngramas(val_ent: str, val_pal: str, indice: int, n: int):
    """Función que genera la lista de todas las palabras del conjunto
    de datos y obtiene la frecuencia de cada una por artículo, 
    especifica a que artículo pertenece y si es una entidad (1) o no
    (0).

    Args:
        val_ent (str): cadena de entidades obtenida en el procesamiento
        val_pal (str): cadena de palabras obtenida en el procesamiento
        indice (int): indice del artículo al que corresponden las cadenas
        n (int): tamaño de la subsecuencia del n-grama

    Returns:
        pd.DataFrame: DataFrame con el indice, la palabra, frecuencia de
        aparición, ID del artículo al que pertenece. Si es solo una palabra
        se incluye la columna de entidad que indica si lo es o no
    """
    if type(val_ent) == float:
        entidades = {}
    else:
        entidades = set(val_ent.split(', '))

    palabras = val_pal.split(', ')
    ngrams = list(nltk.ngrams(palabras, n))
    freq_pal = dict(nltk.FreqDist(ngrams))

    if n == 1:
        lista = []
        for key, value in freq_pal.items():
            word = ", ".join(list(key))
            if word in entidades:
                lista.append([word, value, indice, 1])
            else:
                lista.append([word, value, indice, 0])
        df_frec = pd.DataFrame(
            lista, columns=['Palabra', 'Frecuencia', 'ID_Articulo', 'Entidad'])
    return df_frec

In [46]:
df_palabras = pd.DataFrame()
for i in df_curated.index:
    aux_palabras = lista_ngramas(df_curated.loc[i, 'Entidades de Contenido'],
                                 df_curated.loc[i, 'Contenido procesado'], i, 1)
    df_palabras = pd.concat([df_palabras, aux_palabras], ignore_index=True)

In [50]:
df_palabras.to_csv('../data/curated/palabras.csv', encoding='utf-8-sig')

In [ ]:
entidades = set(df_curated.loc[0, 'Entidades de Contenido'].split(', '))

palabras = df_curated.loc[0, 'Contenido procesado'].split(', ')
ngrams = list(nltk.ngrams(palabras, 2))
freq_pal = dict(nltk.FreqDist(ngrams))
freq_pal

{('movimiento', 'dólar'): 1,
 ('dólar', 'semanas'): 1,
 ('semanas', 'atrás'): 1,
 ('atrás', 'venía'): 1,
 ('venía', 'mostrando'): 1,
 ('mostrando', 'comportamientos'): 1,
 ('comportamientos', 'bajistas'): 1,
 ('bajistas', 'constantes'): 1,
 ('constantes', 'viernes'): 1,
 ('viernes', 'enero'): 2,
 ('enero', 'sorprendió'): 1,
 ('sorprendió', 'leve'): 1,
 ('leve', 'repunte'): 1,
 ('repunte', 'presentó'): 1,
 ('presentó', 'miércoles'): 1,
 ('miércoles', 'febrero'): 1,
 ('febrero', 'fecha'): 1,
 ('fecha', 'comenzó'): 1,
 ('comenzó', 'caer'): 1,
 ('caer', 'nuevamente'): 1,
 ('nuevamente', 'comportamiento'): 1,
 ('comportamiento', 'duró'): 1,
 ('duró', 'volatilidad'): 1,
 ('volatilidad', 'moneda'): 1,
 ('moneda', 'viene'): 1,
 ('viene', 'acompañada'): 1,
 ('acompañada', 'serie'): 1,
 ('serie', 'sucesos'): 1,
 ('sucesos', 'nacionales'): 1,
 ('nacionales', 'internacionales'): 1,
 ('internacionales', 'semana'): 1,
 ('semana', 'excepción'): 1,
 ('excepción', 'evento'): 1,
 ('evento', 'clave'): 1,

In [ ]:
'hola' in 'hola cómo estas'

True

In [ ]:
entidades = set(df_curated.loc[0, 'Entidades de Contenido'].split(', '))

palabras = df_curated.loc[0, 'Contenido procesado'].split(', ')
ngrams = list(nltk.ngrams(palabras, 2))
freq_pal = nltk.FreqDist(ngrams)
freq_pal = dict(nltk.FreqDist(palabras))

lista = []
for key, value in freq_pal.items():
    if key in entidades:
        lista.append([key, value, indice, 1])
    else:
        lista.append([key, value, indice, 0])

return pd.DataFrame(lista, columns=['Palabra', 'Frecuencia', 'ID_Articulo', 'Entidad'])
